In [1]:
# %pip install fuzzywuzzy

In [ ]:
import numpy as np
import pandas as pd
from ast import literal_eval
import re
import string

In [3]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("trolukovich/nutritional-values-for-common-foods-and-products")

# print("Path to dataset files:", path)

In [6]:
df_recipes = pd.read_csv("./data/recipes_ingredients.csv", nrows=1000)
df_nutritions = pd.read_csv("./data/nutrition.csv")
df_food = pd.read_csv("./data/food.csv")

In [7]:
df_recipes.head()

,id,name,description,ingredients,ingredients_raw,steps,servings,serving_size,tags
0,71247,Cherry Streusel Cobbler,"I haven't made this in years, so I'm just gues...","[""cherry pie filling"", ""condensed milk"", ""melt...","[""2 (21 ounce) cans cherry pie filling"",""2...","[""Preheat oven to 375°F."", ""Spread cherry pie ...",6.0,1 (347 g),"[""60-minutes-or-less"", ""time-to-make"", ""course..."
1,76133,Reuben and Swiss Casserole Bake,I think this is even better than a reuben sand...,"[""corned beef chopped"", ""sauerkraut cold water...","[""1/2-1 lb corned beef, cooked and choppe...","[""Set oven to 350 degrees F."", ""Butter a 9 x 1...",4.0,1 (207 g),"[""60-minutes-or-less"", ""time-to-make"", ""course..."
2,503816,Yam-Pecan Recipe,A lady I work with heard me taking about ZWT a...,"[""unsalted butter"", ""vegetable oil"", ""all - pu...","[""3/4 cup unsalted butter, at room tempera...","[""Preheat oven to 350°F In a mixing bowl, usi...",8.0,1 (198 g),"[""time-to-make"", ""course"", ""main-ingredient"", ..."
3,418749,Tropical Orange Layer Cake,An easy and delicious cake. Great for a summ...,"[""orange cake mix"", ""instant vanilla pudding"",...","[""1 (18 ounce) pkge.orange cake mix"",""1 (3...","[""In a large mixing bowl, combine the first 6 ...",16.0,1 (191 g),"[""60-minutes-or-less"", ""time-to-make"", ""course..."
4,392934,Safe to Eat Raw Chocolate Chip Oreo Cookie &qu...,I was searching the web for something like thi...,"[""butter"", ""brown sugar"", ""granulated sugar"", ...","[""1/2 cup butter, room temperature "",""1/2 ...","[""Cream butter and sugars together."", ""Blend i...",24.0,1 (26 g),"[""15-minutes-or-less"", ""time-to-make"", ""course..."


In [8]:
# Since the description section just contains some comment about the recipe, i.e. not very informative.
# So we will leave out the description column for now.
df_recipes.drop("description", axis=1, inplace=True)

In [9]:
# Missing values
num_recipe_with_missing_info = df_recipes.isnull().any(axis=1).sum()
print(f"Number of recipes with missing info: {num_recipe_with_missing_info}")
missing_per_column_recipe = df_recipes.isnull().sum()
print(f"Missing values per column:\n{missing_per_column_recipe}")

# Since there are only 4 recipes with missing info, take them out
df_recipes.dropna(inplace=True)

Number of recipes with missing info: 1
Missing values per column:
id                 0
name               0
ingredients        0
ingredients_raw    0
steps              0
servings           1
serving_size       0
tags               0
dtype: int64


In [10]:
df_recipes.head()

,id,name,ingredients,ingredients_raw,steps,servings,serving_size,tags
0,71247,Cherry Streusel Cobbler,"[""cherry pie filling"", ""condensed milk"", ""melt...","[""2 (21 ounce) cans cherry pie filling"",""2...","[""Preheat oven to 375°F."", ""Spread cherry pie ...",6.0,1 (347 g),"[""60-minutes-or-less"", ""time-to-make"", ""course..."
1,76133,Reuben and Swiss Casserole Bake,"[""corned beef chopped"", ""sauerkraut cold water...","[""1/2-1 lb corned beef, cooked and choppe...","[""Set oven to 350 degrees F."", ""Butter a 9 x 1...",4.0,1 (207 g),"[""60-minutes-or-less"", ""time-to-make"", ""course..."
2,503816,Yam-Pecan Recipe,"[""unsalted butter"", ""vegetable oil"", ""all - pu...","[""3/4 cup unsalted butter, at room tempera...","[""Preheat oven to 350°F In a mixing bowl, usi...",8.0,1 (198 g),"[""time-to-make"", ""course"", ""main-ingredient"", ..."
3,418749,Tropical Orange Layer Cake,"[""orange cake mix"", ""instant vanilla pudding"",...","[""1 (18 ounce) pkge.orange cake mix"",""1 (3...","[""In a large mixing bowl, combine the first 6 ...",16.0,1 (191 g),"[""60-minutes-or-less"", ""time-to-make"", ""course..."
4,392934,Safe to Eat Raw Chocolate Chip Oreo Cookie &qu...,"[""butter"", ""brown sugar"", ""granulated sugar"", ...","[""1/2 cup butter, room temperature "",""1/2 ...","[""Cream butter and sugars together."", ""Blend i...",24.0,1 (26 g),"[""15-minutes-or-less"", ""time-to-make"", ""course..."


In [11]:
df_nutritions.head()

,Unnamed: 0,name,serving_size,calories,total_fat,saturated_fat,cholesterol,sodium,choline,folate,...,fat,saturated_fatty_acids,monounsaturated_fatty_acids,polyunsaturated_fatty_acids,fatty_acids_total_trans,alcohol,ash,caffeine,theobromine,water
0,0,Cornstarch,100 g,381,0.1g,NaN,0,9.00 mg,0.4 mg,0.00 mcg,...,0.05 g,0.009 g,0.016 g,0.025 g,0.00 mg,0.0 g,0.09 g,0.00 mg,0.00 mg,8.32 g
1,1,"Nuts, pecans",100 g,691,72g,6.2g,0,0.00 mg,40.5 mg,22.00 mcg,...,71.97 g,6.180 g,40.801 g,21.614 g,0.00 mg,0.0 g,1.49 g,0.00 mg,0.00 mg,3.52 g
2,2,"Eggplant, raw",100 g,25,0.2g,NaN,0,2.00 mg,6.9 mg,22.00 mcg,...,0.18 g,0.034 g,0.016 g,0.076 g,0.00 mg,0.0 g,0.66 g,0.00 mg,0.00 mg,92.30 g
3,3,"Teff, uncooked",100 g,367,2.4g,0.4g,0,12.00 mg,13.1 mg,0,...,2.38 g,0.449 g,0.589 g,1.071 g,0,0,2.37 g,0,0,8.82 g
4,4,"Sherbet, orange",100 g,144,2g,1.2g,1mg,46.00 mg,7.7 mg,4.00 mcg,...,2.00 g,1.160 g,0.530 g,0.080 g,1.00 mg,0.0 g,0.40 g,0.00 mg,0.00 mg,66.10 g


In [12]:
# Missing values
num_nutrition_with_missing_info = df_nutritions.isnull().any(axis=1).sum()
print(f"Number of recipes with missing info: {num_nutrition_with_missing_info}")
missing_per_column_nutrition = df_nutritions.isnull().sum()
missing_per_column_nutrition = missing_per_column_nutrition[missing_per_column_nutrition > 0]
print(f"Missing values per column:\n{missing_per_column_nutrition}")

Number of recipes with missing info: 1590
Missing values per column:
saturated_fat    1590
dtype: int64


In [13]:
# Since the most important columns that we are looking at are:
# name and the macro-nutrients: calories, fat, protein, and carbohydrate
# Lucky they are all present for every nutrient in this dataset.
# All the missing entries are from "saturated_fat", we will just fill in 0's, since they are probably in a insignificant amount.
df_nutritions.fillna(0, inplace=True)

In [14]:
# We need to combine the 2 datasets to fill in the total nutritional info for each recipe

def parse_ingredient_list(ingredient_str):
    try:
        return literal_eval(ingredient_str)
    except (ValueError, SyntaxError):
        # Return None or an empty list if parsing fails
        return []

In [15]:
# First, we ensure the "ingredient" and "ingredient_raw" columns are parsed as lists, instead of a string
df_recipes['ingredients'] = df_recipes['ingredients'].apply(parse_ingredient_list)
df_recipes['ingredients_raw'] = df_recipes['ingredients_raw'].apply(parse_ingredient_list)
df_recipes = df_recipes.dropna(subset=['ingredients'])
df_recipes = df_recipes.dropna(subset=['ingredients_raw'])

In [16]:
df_recipes.head()

,id,name,ingredients,ingredients_raw,steps,servings,serving_size,tags
0,71247,Cherry Streusel Cobbler,"[cherry pie filling, condensed milk, melted ma...","[2 (21 ounce) cans cherry pie filling, 2 ...","[""Preheat oven to 375°F."", ""Spread cherry pie ...",6.0,1 (347 g),"[""60-minutes-or-less"", ""time-to-make"", ""course..."
1,76133,Reuben and Swiss Casserole Bake,"[corned beef chopped, sauerkraut cold water, s...","[1/2-1 lb corned beef, cooked and chopped...","[""Set oven to 350 degrees F."", ""Butter a 9 x 1...",4.0,1 (207 g),"[""60-minutes-or-less"", ""time-to-make"", ""course..."
2,503816,Yam-Pecan Recipe,"[unsalted butter, vegetable oil, all - purpose...","[3/4 cup unsalted butter, at room temperat...","[""Preheat oven to 350°F In a mixing bowl, usi...",8.0,1 (198 g),"[""time-to-make"", ""course"", ""main-ingredient"", ..."
3,418749,Tropical Orange Layer Cake,"[orange cake mix, instant vanilla pudding, ora...","[1 (18 ounce) pkge.orange cake mix, 1 (3 ...","[""In a large mixing bowl, combine the first 6 ...",16.0,1 (191 g),"[""60-minutes-or-less"", ""time-to-make"", ""course..."
4,392934,Safe to Eat Raw Chocolate Chip Oreo Cookie &qu...,"[butter, brown sugar, granulated sugar, milk, ...","[1/2 cup butter, room temperature , 1/2 c...","[""Cream butter and sugars together."", ""Blend i...",24.0,1 (26 g),"[""15-minutes-or-less"", ""time-to-make"", ""course..."


In [17]:
for i in range(5):
  print(df_recipes['ingredients'][i])
  print(df_recipes['ingredients_raw'][i])

['cherry pie filling', 'condensed milk', 'melted margarine', 'cinnamon', 'nutmeg', 'light brown sugar', 'flour', 'margarine', 'chopped nuts', 'oats', 'butter - flavored cooking spray']
['2 (21   ounce) cans   cherry pie filling', '2       eggs', '1 (14   ounce) can   sweetened condensed milk (not evaporated)', '1/4  cup   melted margarine', '1/2  teaspoon    cinnamon', '1/4  teaspoon    nutmeg', '1/2  cup   firmly packed light brown sugar', '1/2  cup    self-rising flour', '1/4  cup    margarine', "1/2  cup   chopped nuts (chef's choice)", '1/2  cup    quick-cooking oats', '  butter-flavored cooking spray']
['corned beef chopped', 'sauerkraut cold water', 'swiss cheese shredded', 'rye bread', 'butter']
['1/2-1   lb    corned beef, cooked and chopped ', '1/4  cup    thousand island dressing (Kraft is best)', '1 (16   ounce) can   sauerkraut, drained and rinsed with cold water ', '1/2  lb    swiss cheese, shredded ', '6   slices    rye bread, crumbled into very small pieces ', '1/4  cup 

In [18]:
# Function to extract quantity and unit from a raw ingredient string
def extract_quantity_unit(ingredient, raw_ingredient):
    result = raw_ingredient.replace(ingredient, '').strip()
    number = re.search(r'\d+/\d+|\d+', result)
    unit = re.search(r'ounce|lb|cup|tablespoon|teaspoon', result)

    res = []

    if not number or not unit:
      return None
    else:
      number = number.group(0)
      unit = unit.group(0)
      return [number, unit]


# Function to find the best match in the raw ingredients and extract the quantity
def match_ingredient_with_quantity(ingredients, ingredients_raw):
    matched_ingredients = []
    for ingredient in ingredients:
        # Try to find a match in ingredients_raw based on partial matching
        for raw in ingredients_raw:
            if ingredient in raw:
                if extract_quantity_unit(ingredient,raw):
                  matched_ingredients.append([extract_quantity_unit(ingredient,raw), ingredient])
                break

    return matched_ingredients

df_recipes['matched_ingredients'] = df_recipes.apply(lambda row: match_ingredient_with_quantity(row['ingredients'], row['ingredients_raw']), axis=1)

In [19]:
print(df_recipes['matched_ingredients'])

0      [[[2, ounce], cherry pie filling], [[1, ounce]...
1                                 [[[1/4, cup], butter]]
2      [[[3/4, cup], unsalted butter], [[1/2, cup], v...
3      [[[1, ounce], orange cake mix], [[1, ounce], i...
4      [[[1/2, cup], butter], [[1/2, cup], brown suga...
                             ...                        
995                         [[[1/3, cup], orange juice]]
996    [[[1, cup], dried soybeans], [[2, cup], water]...
997                                                   []
998    [[[1, ounce], chicken broth], [[1, ounce], fro...
999                                                   []
Name: matched_ingredients, Length: 999, dtype: object


In [20]:
# now we convert all other units into values in unit (100g)
# TODO: further adjust these values according to the different kind of ingredients
convertion = {"ounce": 28.35, "lb": 453.59, "cup": 236.59, "tablespoon": 14.79, "teaspoon": 4.93}

from fractions import Fraction

def convert_to_g(matched_ingredients):
  res = []
  for quantity, ingredient in matched_ingredients:
    if quantity[1] in convertion:
      quantity_in_g = float(Fraction(quantity[0])) * convertion[quantity[1]]
      res.append([quantity_in_g, ingredient])
  return res

df_recipes['matched_ingredients'] = df_recipes['matched_ingredients'].apply(convert_to_g)

In [21]:
# print(df_recipes['matched_ingredients'][100])

In [22]:
df_recipes.head()

,id,name,ingredients,ingredients_raw,steps,servings,serving_size,tags,matched_ingredients
0,71247,Cherry Streusel Cobbler,"[cherry pie filling, condensed milk, melted ma...","[2 (21 ounce) cans cherry pie filling, 2 ...","[""Preheat oven to 375°F."", ""Spread cherry pie ...",6.0,1 (347 g),"[""60-minutes-or-less"", ""time-to-make"", ""course...","[[56.7, cherry pie filling], [28.35, condensed..."
1,76133,Reuben and Swiss Casserole Bake,"[corned beef chopped, sauerkraut cold water, s...","[1/2-1 lb corned beef, cooked and chopped...","[""Set oven to 350 degrees F."", ""Butter a 9 x 1...",4.0,1 (207 g),"[""60-minutes-or-less"", ""time-to-make"", ""course...","[[59.1475, butter]]"
2,503816,Yam-Pecan Recipe,"[unsalted butter, vegetable oil, all - purpose...","[3/4 cup unsalted butter, at room temperat...","[""Preheat oven to 350°F In a mixing bowl, usi...",8.0,1 (198 g),"[""time-to-make"", ""course"", ""main-ingredient"", ...","[[177.4425, unsalted butter], [118.295, vegeta..."
3,418749,Tropical Orange Layer Cake,"[orange cake mix, instant vanilla pudding, ora...","[1 (18 ounce) pkge.orange cake mix, 1 (3 ...","[""In a large mixing bowl, combine the first 6 ...",16.0,1 (191 g),"[""60-minutes-or-less"", ""time-to-make"", ""course...","[[28.35, orange cake mix], [28.35, instant van..."
4,392934,Safe to Eat Raw Chocolate Chip Oreo Cookie &qu...,"[butter, brown sugar, granulated sugar, milk, ...","[1/2 cup butter, room temperature , 1/2 c...","[""Cream butter and sugars together."", ""Blend i...",24.0,1 (26 g),"[""15-minutes-or-less"", ""time-to-make"", ""course...","[[118.295, butter], [118.295, brown sugar], [5..."


In [23]:
# Experimenting with sample data
df1 = pd.DataFrame({
    'recipe_id': [1, 2],
    'match_ingredients': [
        [[13993.707025, 'olive oil'], [218.74409999999997, 'fresh basil'],
         [48.60979999999999, 'Tabasco sauce'], [13993.707025, 'parmesan cheese']],
        [[50, 'corn oil'], [150, 'fresh basil'], [100, 'garlic'], [120, 'parmesan cheese']]
    ]
})

df2 = pd.DataFrame({
    'ingredient': ['corn oil', 'fresh basil', 'Tabasco sauce', 'parmesan cheese', 'oil'],
    'calories': [884, 22, 0, 431, 900],
    'protein': [0, 2.6, 0, 37, 0],
    'fat': [100, 0.5, 0, 29, 100],
    'carbs': [0, 4.4, 0, 4, 0]
})

In [24]:
df1

,recipe_id,match_ingredients
0,1,"[[13993.707025, olive oil], [218.7440999999999..."
1,2,"[[50, corn oil], [150, fresh basil], [100, gar..."


In [25]:
df2

,ingredient,calories,protein,fat,carbs
0,corn oil,884,0.0,100.0,0.0
1,fresh basil,22,2.6,0.5,4.4
2,Tabasco sauce,0,0.0,0.0,0.0
3,parmesan cheese,431,37.0,29.0,4.0
4,oil,900,0.0,100.0,0.0


In [26]:
# Flatten the match_ingredients list into a DataFrame
ingredients_df = df1.explode('match_ingredients').reset_index(drop=True)
ingredients_df[['quantity', 'ingredient']] = pd.DataFrame(ingredients_df['match_ingredients'].tolist(), index=ingredients_df.index)
ingredients_df

,recipe_id,match_ingredients,quantity,ingredient
0,1,"[13993.707025, olive oil]",13993.707025,olive oil
1,1,"[218.74409999999997, fresh basil]",218.744100,fresh basil
2,1,"[48.60979999999999, Tabasco sauce]",48.609800,Tabasco sauce
3,1,"[13993.707025, parmesan cheese]",13993.707025,parmesan cheese
4,2,"[50, corn oil]",50.000000,corn oil
5,2,"[150, fresh basil]",150.000000,fresh basil
6,2,"[100, garlic]",100.000000,garlic
7,2,"[120, parmesan cheese]",120.000000,parmesan cheese


In [27]:
from fuzzywuzzy import process
# Function to get the best match from df2 for each ingredient
def get_best_match(ingredient, choices, threshold=80):
    best_match, score = process.extractOne(ingredient, choices)
    if score >= threshold:
        return best_match
    return None  # If no good match is found


/opt/anaconda3/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [28]:
# Apply fuzzy matching to find the closest ingredient in df2
ingredients_df['matched_ingredient'] = ingredients_df['ingredient'].apply(
    lambda x: get_best_match(x, df2['ingredient'].tolist())
)
ingredients_df

,recipe_id,match_ingredients,quantity,ingredient,matched_ingredient
0,1,"[13993.707025, olive oil]",13993.707025,olive oil,oil
1,1,"[218.74409999999997, fresh basil]",218.744100,fresh basil,fresh basil
2,1,"[48.60979999999999, Tabasco sauce]",48.609800,Tabasco sauce,Tabasco sauce
3,1,"[13993.707025, parmesan cheese]",13993.707025,parmesan cheese,parmesan cheese
4,2,"[50, corn oil]",50.000000,corn oil,corn oil
5,2,"[150, fresh basil]",150.000000,fresh basil,fresh basil
6,2,"[100, garlic]",100.000000,garlic,None
7,2,"[120, parmesan cheese]",120.000000,parmesan cheese,parmesan cheese


In [29]:
# Merge with nutrition DataFrame (df2) using the best match ingredient
merged_df = pd.merge(ingredients_df, df2, left_on='matched_ingredient', right_on='ingredient', how='left')

# Multiply the quantities by the nutrition values and sum by recipe_id
nutrition_columns = ['calories', 'protein', 'fat', 'carbs']
for col in nutrition_columns:
    merged_df[col] = merged_df[col] * merged_df['quantity']

# Aggregate by recipe_id to calculate total nutrition
total_nutrition = merged_df.groupby('recipe_id')[nutrition_columns].sum().reset_index()

# Display total nutrition for each recipe
df1 = df1.merge(total_nutrition, on='recipe_id', how='left')

print(df1[['recipe_id', 'calories', 'protein', 'fat', 'carbs']])

   recipe_id      calories        protein           fat        carbs
0          1  1.863044e+07  518335.894585  1.805298e+06  56937.30214
1          2  9.922000e+04    4830.000000  8.555000e+03   1140.00000


In [30]:
# Using real data
df_recipes.head()


,id,name,ingredients,ingredients_raw,steps,servings,serving_size,tags,matched_ingredients
0,71247,Cherry Streusel Cobbler,"[cherry pie filling, condensed milk, melted ma...","[2 (21 ounce) cans cherry pie filling, 2 ...","[""Preheat oven to 375°F."", ""Spread cherry pie ...",6.0,1 (347 g),"[""60-minutes-or-less"", ""time-to-make"", ""course...","[[56.7, cherry pie filling], [28.35, condensed..."
1,76133,Reuben and Swiss Casserole Bake,"[corned beef chopped, sauerkraut cold water, s...","[1/2-1 lb corned beef, cooked and chopped...","[""Set oven to 350 degrees F."", ""Butter a 9 x 1...",4.0,1 (207 g),"[""60-minutes-or-less"", ""time-to-make"", ""course...","[[59.1475, butter]]"
2,503816,Yam-Pecan Recipe,"[unsalted butter, vegetable oil, all - purpose...","[3/4 cup unsalted butter, at room temperat...","[""Preheat oven to 350°F In a mixing bowl, usi...",8.0,1 (198 g),"[""time-to-make"", ""course"", ""main-ingredient"", ...","[[177.4425, unsalted butter], [118.295, vegeta..."
3,418749,Tropical Orange Layer Cake,"[orange cake mix, instant vanilla pudding, ora...","[1 (18 ounce) pkge.orange cake mix, 1 (3 ...","[""In a large mixing bowl, combine the first 6 ...",16.0,1 (191 g),"[""60-minutes-or-less"", ""time-to-make"", ""course...","[[28.35, orange cake mix], [28.35, instant van..."
4,392934,Safe to Eat Raw Chocolate Chip Oreo Cookie &qu...,"[butter, brown sugar, granulated sugar, milk, ...","[1/2 cup butter, room temperature , 1/2 c...","[""Cream butter and sugars together."", ""Blend i...",24.0,1 (26 g),"[""15-minutes-or-less"", ""time-to-make"", ""course...","[[118.295, butter], [118.295, brown sugar], [5..."


In [31]:
print(type(df_recipes['matched_ingredients'][0]))

<class 'list'>


In [32]:
df_food.head()

,Category,Description,Nutrient Data Bank Number,Data.Alpha Carotene,Data.Ash,Data.Beta Carotene,Data.Beta Cryptoxanthin,Data.Carbohydrate,Data.Cholesterol,Data.Choline,...,Data.Major Minerals.Potassium,Data.Major Minerals.Sodium,Data.Major Minerals.Zinc,Data.Vitamins.Vitamin A - IU,Data.Vitamins.Vitamin A - RAE,Data.Vitamins.Vitamin B12,Data.Vitamins.Vitamin B6,Data.Vitamins.Vitamin C,Data.Vitamins.Vitamin E,Data.Vitamins.Vitamin K
0,BUTTER,"BUTTER,WITH SALT",1001,0,2.11,158,0,0.06,215,19,...,24,576,0.09,2499,684,0.17,0.003,0.0,2.32,7.0
1,BUTTER,"BUTTER,WHIPPED,WITH SALT",1002,0,2.11,158,0,0.06,219,19,...,26,827,0.05,2499,684,0.13,0.003,0.0,2.32,7.0
2,BUTTER OIL,"BUTTER OIL,ANHYDROUS",1003,0,0.00,193,0,0.00,256,22,...,5,2,0.01,3069,840,0.01,0.001,0.0,2.80,8.6
3,CHEESE,"CHEESE,BLUE",1004,0,5.11,74,0,2.34,75,15,...,256,1395,2.66,763,198,1.22,0.166,0.0,0.25,2.4
4,CHEESE,"CHEESE,BRICK",1005,0,3.18,76,0,2.79,94,15,...,136,560,2.60,1080,292,1.26,0.065,0.0,0.26,2.5


In [33]:
df_food_mean = df_food.groupby('Category')[['Data.Carbohydrate', 'Data.Protein', 'Data.Fat.Total Lipid']].mean(numeric_only=True).reset_index()
df_food_mean

,Category,Data.Carbohydrate,Data.Protein,Data.Fat.Total Lipid
0,ABALONE,8.530000,18.365000,3.770000
1,ABIYUCH,17.600000,1.500000,0.100000
2,ACEROLA,7.690000,0.400000,0.300000
3,ACEROLA JUICE,4.800000,0.400000,0.300000
4,ACORN FLOUR,54.650000,7.490000,30.170000
...,...,...,...,...
1178,YARDLONG BEAN,8.900000,2.620000,0.200000
1179,YARDLONG BNS,34.696667,13.636667,0.736667
1180,YAUTIA (TANNIER),23.630000,1.460000,0.400000
1181,YELLOWTAIL,0.000000,26.405000,5.980000


In [ ]:
# Ensure that 'matched_ingredients' are either lists or empty lists
df_recipes['matched_ingredients'] = df_recipes['matched_ingredients'].apply(
    lambda x: x if isinstance(x, list) else ([x] if x else [])
)

# Explode the 'matched_ingredients' column safely
df_recipes_flattened = df_recipes.explode('matched_ingredients').reset_index(drop=True)

# Filter out any rows where 'matched_ingredients' is an empty list or any other invalid type
df_recipes_flattened = df_recipes_flattened[df_recipes_flattened['matched_ingredients'].notna()]

# Split 'matched_ingredients' into 'quantity' and 'ingredient'
df_recipes_flattened[['quantity', 'ingredient']] = pd.DataFrame(df_recipes_flattened['matched_ingredients'].tolist(), index=df_recipes_flattened.index)

# Apply fuzzy matching to find the closest ingredient in df2
df_recipes_flattened['matched_ingredient'] = df_recipes_flattened['ingredient'].apply(
    lambda x: get_best_match(x, df_food_mean['Category'].tolist())
)

# Display the result
df_recipes_flattened

In [ ]:
# from fuzzywuzzy import fuzz
# from fuzzywuzzy import process
# from functools import lru_cache

# # Ensure that matched_ingredients values are lists
# df_recipes['matched_ingredients'] = df_recipes['matched_ingredients'].map(lambda x: x if isinstance(x, list) else [x])

# # Flatten 'matched_ingredients' column with DataFrame and stack for speed
# df_recipes_flattened = (
#     pd.DataFrame({
#         'recipe_id': df_recipes.index.repeat(df_recipes['matched_ingredients'].str.len()),
#         'matched_ingredients': [ingredient for sublist in df_recipes['matched_ingredients'] for ingredient in sublist]
#     })
# )

# # Split 'matched_ingredients' into 'quantity' and 'ingredient' columns directly
# df_recipes_flattened[['quantity', 'ingredient']] = pd.DataFrame(df_recipes_flattened['matched_ingredients'].tolist(), index=df_recipes_flattened.index)

# # Create a cache to speed up repeated lookups for fuzzy matching
# @lru_cache(maxsize=10000)
# def get_best_match(ingredient, choices):
#     return process.extractOne(ingredient, choices, scorer=fuzz.partial_ratio)[0]

# # Apply fuzzy matching with caching
# df_recipes_flattened['matched_ingredient'] = df_recipes_flattened['ingredient'].map(lambda x: get_best_match(x, tuple(df_nutritions['name'])))

# # Display the result
# print(df_recipes_flattened)

In [ ]:
# Save DataFrame to CSV
df_recipes_flattened.to_csv('recipes_flattened.csv', index=False)  # index=False omits the row index from the file


In [ ]:
df_recipes_flattened

In [ ]:
# df_nutritions.head()

In [ ]:
# Merge with nutrition DataFrame df_nutritions using the best match ingredient
merged_df = pd.merge(df_recipes_flattened, df_food_mean, left_on='matched_ingredient', right_on='Category', how='left')

# Multiply the quantities by the nutrition values and sum by recipe_id
nutrition_columns = ['Data.Carbohydrate', 'Data.Protein', 'Data.Fat.Total Lipid']
for col in nutrition_columns:
    merged_df[col] = merged_df[col] * merged_df['quantity']

# Aggregate by recipe_id to calculate total nutrition
total_nutrition = merged_df.groupby('id')[nutrition_columns].sum().reset_index()

# Display total nutrition for each recipe
df_recipes = df_recipes.merge(total_nutrition, on='id', how='left')

# print(df_recipes[['Data.Carbohydrate', 'Data.Protein', 'Data.Fat.Total Lipid']])
df_recipes

In [ ]:
df_recipes.to_csv('processed_recipes.csv', index=False)

In [ ]:
# drop any Nan values
df_recipes.dropna(inplace=True)
df_recipes